In [1]:
import numpy as np
import networkx as nx
import copy
import pickle
import sys
import time
sys.path.append("/home/ubuntu/CE_scheme/")
from BayesCard.Evaluation.utils import parse_query
from Schemas.stats.schema import gen_stats_light_schema
from Join_scheme.data_prepare import identify_key_values, process_stats_data

In [12]:
data_path = "/home/ubuntu/End-to-End-CardEst-Benchmark/datasets/stats_simplified/{}.csv"
schema = gen_stats_light_schema(data_path)
all_keys, equivalent_keys = identify_key_values(schema)
print(equivalent_keys)
model_path = "/home/ubuntu/data_CE/CE_scheme_models/model_stats_greedy_200.pkl"
with open(model_path, "rb") as f:
    new_BE = pickle.load(f)

{'posts.Id': {'tags.ExcerptPostId', 'postHistory.PostId', 'postLinks.PostId', 'comments.PostId', 'posts.Id', 'postLinks.RelatedPostId', 'votes.PostId'}, 'users.Id': {'users.Id', 'votes.UserId', 'posts.OwnerUserId', 'postHistory.UserId', 'badges.UserId', 'comments.UserId'}}


In [13]:
import numpy as np
import copy

from Join_scheme.join_graph import process_condition, get_join_hyper_graph
from Join_scheme.data_prepare import identify_key_values
from BayesCard.Evaluation.cardinality_estimation import timestamp_transorform, construct_table_query


class Factor:
    """
    This the class defines a multidimensional conditional probability.
    """
    def __init__(self, variables, pdfs, equivalent_variables=[]):
        self.variables = variables
        self.equivalent_variables = equivalent_variables
        self.pdfs = pdfs
        self.cardinalities = dict()
        for i, var in enumerate(self.variables):
            self.cardinalities[var] = pdfs.shape[i]
            if len(equivalent_variables) != 0:
                self.cardinalities[equivalent_variables[i]] = pdfs.shape[i]


class Bound_ensemble:
    """
    This the class where we store all the trained models and perform inference on the bound.
    """
    def __init__(self, bns, table_buckets, schema):
        self.bns = bns
        self.table_buckets = table_buckets
        self.schema = schema
        self.all_keys, self.equivalent_keys = identify_key_values(schema)

    def parse_query_simple(self, query):
        """
        If your selection query contains no aggregation and nested sub-queries, you can use this function to parse a
        join query. Otherwise, use parse_query function.
        """
        query = query.replace(" where ", " WHERE ")
        query = query.replace(" from ", " FROM ")
        query = query.replace(" and ", " AND ")
        query = query.split(";")[0]
        query = query.strip()
        tables_all = {}
        join_cond = []
        table_query = {}
        join_keys = {}
        tables_str = query.split(" WHERE ")[0].split(" FROM ")[-1]
        for table_str in tables_str.split(","):
            table_str = table_str.strip()
            if " as " in table_str:
                tables_all[table_str.split(" as ")[-1]] = table_str.split(" as ")[0]
            else:
                tables_all[table_str.split(" ")[-1]] = table_str.split(" ")[0]

        # processing conditions
        conditions = query.split(" WHERE ")[-1].split(" AND ")
        for cond in conditions:
            table, cond, join, join_key = process_condition(cond, tables_all)
            if not join:
                attr = cond[0]
                op = cond[1]
                value = cond[2]
                if "Date" in attr:
                    assert "::timestamp" in value
                    value = timestamp_transorform(value.strip().split("::timestamp")[0])
                if table not in table_query:
                    table_query[table] = dict()
                construct_table_query(self.bns[table], table_query[table], attr, op, value)
            else:
                join_cond.append(cond)
                for tab in join_key:
                    if tab in join_keys:
                        join_keys[tab].add(join_key[tab])
                    else:
                        join_keys[tab] = set([join_key[tab]])

        return tables_all, table_query, join_cond, join_keys

    def get_all_id_conidtional_distribution(self, table_queries, join_keys, equivalent_group):
        res = dict()
        for table in join_keys:
            key_attrs = list(join_keys[table])
            if table in table_queries:
                table_query = table_queries[table]
            else:
                table_query = {}
            id_attrs, probs = self.bns[table].query_id_prob(table_query, key_attrs)
            new_id_attrs = []
            for K in id_attrs:
                for PK in equivalent_group:
                    if K in equivalent_group[PK]:
                        new_id_attrs.append(PK)
            assert len(new_id_attrs) == len(id_attrs)
            res[table] = Factor(id_attrs, probs, new_id_attrs)
        return res

    def eliminate_one_key_group_general(self, tables, key_group, factors):
        rest_groups = dict()
        rest_group_tables = dict()
        for table in tables:
            assert key_group in factors[table].equivalent_variables
            temp = copy.deepcopy(factors[table].equivalent_variables)
            temp.remove(key_group)
            for keys in temp:
                if keys in rest_groups:
                    assert factors[table].cardinalities[keys] == rest_groups[keys]
                    rest_group_tables[keys].append(table)
                else:
                    rest_groups[keys] = factors[table].cardinalities[keys]
                    rest_group_tables[keys] = [table]

        new_factor_variables = []
        new_factor_cardinalities = []
        for key in rest_groups:
            new_factor_variables.append(key)
            new_factor_cardinalities.append(rest_groups[key])
        new_factor_pdf = np.zeros(tuple(new_factor_cardinalities))

    def eliminate_one_key_group(self, tables, key_group, factors, relevant_keys):
        """This version only supports 2D distributions"""
        #print(key_group)
        rest_group = None
        rest_group_cardinalty = 0
        eliminated_tables = []
        rest_group_tables = []
        for table in tables:
            assert key_group in factors[table].equivalent_variables
            temp = copy.deepcopy(factors[table].equivalent_variables)
            temp.remove(key_group)
            if len(temp) == 0:
                eliminated_tables.append(table)
            for key in temp:
                if rest_group:
                    assert factors[table].cardinalities[key] == rest_group_cardinalty
                    rest_group_tables.append(table)
                else:
                    rest_group = key
                    rest_group_cardinalty = factors[table].cardinalities[key]
                    rest_group_tables = [table]

        all_probs_eliminated = []
        all_modes_eliminated = []
        #print(eliminated_tables)
        #print(rest_group_tables)
        for table in eliminated_tables:
            bin_modes = self.table_buckets[table].oned_bin_modes[relevant_keys[key_group][table]]
            all_probs_eliminated.append(factors[table].pdfs)
            all_modes_eliminated.append(np.minimum(bin_modes, factors[table].pdfs))
        if rest_group:
            new_factor_pdf = np.zeros(rest_group_cardinalty)
        else:
            return self.compute_bound_oned(all_probs_eliminated, all_modes_eliminated)
        
        #print(np.sum(factors[rest_group_tables[0]].pdfs))
        #print(np.sum(factors[rest_group_tables[0]].pdfs, axis=1))
        
        #print(self.table_buckets[rest_group_tables[0]].twod_bin_modes[relevant_keys[key_group][rest_group_tables[0]]])
        for i in range(rest_group_cardinalty):
            #print(i)
            rest_group_probs_eliminated = []
            rest_group_modes_eliminated = []
            for table in rest_group_tables:
                
                idx_f = factors[table].equivalent_variables.index(key_group)
                idx_b = self.table_buckets[table].id_attributes.index(relevant_keys[key_group][table])
                #print(idx_f, idx_b)
                bin_modes = self.table_buckets[table].twod_bin_modes[relevant_keys[key_group][table]]
                if idx_f == 0 and idx_b == 0:
                    rest_group_probs_eliminated.append(factors[table].pdfs[:, i])
                    rest_group_modes_eliminated.append(np.minimum(bin_modes[:, i], factors[table].pdfs[:, i]))
                elif idx_f == 0 and idx_b == 1:
                    rest_group_probs_eliminated.append(factors[table].pdfs[:, i])
                    rest_group_modes_eliminated.append(np.minimum(bin_modes[i, :], factors[table].pdfs[:, i]))
                elif idx_f == 1 and idx_b == 0:
                    #print(np.sum(factors[table].pdfs[i, :]))
                    rest_group_probs_eliminated.append(factors[table].pdfs[i, :])
                    rest_group_modes_eliminated.append(np.minimum(bin_modes[:, i], factors[table].pdfs[i, :]))
                else:
                    rest_group_probs_eliminated.append(factors[table].pdfs[i, :])
                    rest_group_modes_eliminated.append(np.minimum(bin_modes[i, :], factors[table].pdfs[i, :]))
            #print("non-zero prob:", len(all_probs_eliminated), np.sum(all_probs_eliminated[-1] != 0), np.sum(all_probs_eliminated[-1]))
            #print("non-zero mode:", len(all_modes_eliminated), np.sum(all_modes_eliminated[-1] != 0), np.sum(all_modes_eliminated[-1]))
            new_factor_pdf[i] = self.compute_bound_oned(all_probs_eliminated + rest_group_probs_eliminated, 
                                                        all_modes_eliminated + rest_group_modes_eliminated)

        for table in rest_group_tables:
            factors[table] = Factor([rest_group], new_factor_pdf, [rest_group])

        return None

    def compute_bound_oned(self, all_probs, all_modes):
        all_probs = np.stack(all_probs, axis=0)
        #print(np.sum(all_probs != 0))
        all_modes = np.stack(all_modes, axis=0)
        #print(np.sum(all_modes != 0))
        multiplier = np.prod(all_modes, axis=0)
        non_zero_idx = np.where(multiplier != 0)[0]
        #print(len(non_zero_idx))
        min_number = np.amin(all_probs[:, non_zero_idx]/all_modes[:, non_zero_idx], axis=0)
        multiplier[non_zero_idx] = multiplier[non_zero_idx] * min_number
        #print("multipler:", np.sum(multiplier))
        return np.sum(multiplier)

    def get_optimal_elimination_order(self, equivalent_group, join_keys, factors):
        cardinalities = dict()
        lengths = dict()
        tables_involved = dict()
        relevant_keys = dict()
        for group in equivalent_group:
            relevant_keys[group] = dict()
            lengths[group] = len(equivalent_group[group])
            cardinalities[group] = []
            tables_involved[group] = set([])
            for keys in equivalent_group[group]:
                for table in join_keys:
                    if keys in join_keys[table]:
                        cardinalities[group].append(len(join_keys[table]))
                        tables_involved[group].add(table)
                        variables = factors[table].variables
                        variables[variables.index(keys)] = group
                        factors[table].variables = variables
                        relevant_keys[group][table] = keys
                        break
            cardinalities[group] = np.asarray(cardinalities[group])

        optimal_order = list(equivalent_group.keys())
        for i in range(len(optimal_order)):
            min_idx = i
            for j in range(i+1, len(optimal_order)):
                min_group = optimal_order[min_idx]
                curr_group = optimal_order[j]
                if np.max(cardinalities[curr_group]) < np.max(cardinalities[min_group]):
                    min_idx = j
                else:
                    min_max_tables = np.max(cardinalities[min_group])
                    min_num_max_tables = len(np.where(cardinalities[min_group] == min_max_tables)[0])
                    curr_max_tables = np.max(cardinalities[curr_group])
                    curr_num_max_tables = len(np.where(cardinalities[curr_group] == curr_max_tables)[0])
                    if curr_num_max_tables < min_num_max_tables:
                        min_idx = j
                    elif lengths[curr_group] < lengths[min_group]:
                        min_idx = j
            optimal_order[i], optimal_order[min_idx] = optimal_order[min_idx], optimal_order[i]
        return optimal_order, tables_involved, relevant_keys

    def get_cardinality_bound(self, query_str):
        tables_all, table_queries, join_cond, join_keys = self.parse_query_simple(query_str)
        equivalent_group = get_join_hyper_graph(join_keys, self.equivalent_keys)
        conditional_factors = self.get_all_id_conidtional_distribution(table_queries, join_keys, equivalent_group)
        optimal_order, tables_involved, relevant_keys = self.get_optimal_elimination_order(equivalent_group, join_keys,
                                                                            conditional_factors)

        for key_group in optimal_order:
            tables = tables_involved[key_group]
            res = self.eliminate_one_key_group(tables, key_group, conditional_factors, relevant_keys)
        return res


In [14]:
BE = Bound_ensemble(new_BE.bns, new_BE.table_buckets, new_BE.schema)

In [15]:
query57 = "SELECT COUNT(*) FROM posts as p, postLinks as pl, postHistory as ph, votes as v, badges as b, users as u WHERE p.Id = pl.RelatedPostId AND u.Id = p.OwnerUserId AND u.Id = b.UserId AND u.Id = ph.UserId AND u.Id = v.UserId AND p.CommentCount>=0 AND p.CommentCount<=13 AND ph.PostHistoryTypeId=5 AND ph.CreationDate<='2014-08-13 09:20:10'::timestamp AND v.CreationDate>='2010-07-19 00:00:00'::timestamp AND b.Date<='2014-09-09 10:24:35'::timestamp AND u.Views>=0 AND u.DownVotes>=0 AND u.CreationDate>='2010-08-04 16:59:53'::timestamp AND u.CreationDate<='2014-07-22 15:15:22'::timestamp;"
t = time.time()
res = BE.get_cardinality_bound(query57)
print(time.time() - t)
print(res)

0.08522915840148926
1211892519680.138


In [16]:
query_file = "/home/ubuntu/End-to-End-CardEst-Benchmark/workloads/stats_CEB/sub_plan_queries/stats_CEB_sub_queries.sql"
with open(query_file, "r") as f:
    queries = f.readlines()


In [17]:
qerror = []
latency = []
pred = []
for i, query_str in enumerate(queries):
    query = query_str.split("||")[0][:-1]
    print("========================")
    true_card = int(query_str.split("||")[-1])
    t = time.time()
    res = BE.get_cardinality_bound(query)
    pred.append(res)
    latency.append(time.time() - t)
    qerror.append(res/true_card)
    print(f"estimating query {i}: predicted {res}, true_card {true_card}, qerror {res/true_card}, latency {time.time() - t}")

estimating query 0: predicted 79851.0, true_card 79851, qerror 1.0, latency 0.011817693710327148
estimating query 1: predicted 12295160.0, true_card 10220614, qerror 1.202976650913536, latency 0.018449783325195312
estimating query 2: predicted 4126975.0, true_card 1458075, qerror 2.830427104229892, latency 0.013134241104125977
estimating query 3: predicted 4242223.095235909, true_card 1709781, qerror 2.4811499807495285, latency 0.013792037963867188
estimating query 4: predicted 10719765.0, true_card 7491903, qerror 1.4308467421428175, latency 0.013337850570678711
estimating query 5: predicted 1540892.1294773642, true_card 428612, qerror 3.5950746350483986, latency 0.014172554016113281
estimating query 6: predicted 61288345.969291344, true_card 55900138, qerror 1.0963898867171193, latency 0.013277769088745117
estimating query 7: predicted 11101.71481716133, true_card 10972, qerror 1.011822349358488, latency 0.009615421295166016
estimating query 8: predicted 60182861.62168815, true_card 

estimating query 68: predicted 47719.918032786896, true_card 29072, qerror 1.6414391178036218, latency 0.004605293273925781
estimating query 69: predicted 691.2295081967213, true_card 433, qerror 1.596372998144853, latency 0.0044994354248046875
estimating query 70: predicted 70851.0, true_card 62294, qerror 1.1373647542299419, latency 0.002527475357055664
estimating query 71: predicted 27267.786885245907, true_card 4582, qerror 5.951066539774314, latency 0.00536656379699707
estimating query 72: predicted 171399.0, true_card 173065, qerror 0.9903735590674024, latency 0.005171298980712891
estimating query 73: predicted 148817.0, true_card 35157, qerror 4.2329265864550445, latency 0.004985809326171875
estimating query 74: predicted 925262.8738420198, true_card 670344, qerror 1.380280682518259, latency 0.010866880416870117
estimating query 75: predicted 11102.0, true_card 11077, qerror 1.0022569287713279, latency 0.004047870635986328
estimating query 76: predicted 261976.47075081436, true_

estimating query 127: predicted 2187293.5976249976, true_card 1486002, qerror 1.471931799301076, latency 0.007640361785888672
estimating query 128: predicted 336826493.0676457, true_card 232039659, qerror 1.4515901916044693, latency 0.010143280029296875
estimating query 129: predicted 3762607.098522448, true_card 368186, qerror 10.219310616162613, latency 0.013293981552124023
estimating query 130: predicted 1542911.07518102, true_card 956304, qerror 1.6134106677176085, latency 0.011561870574951172
estimating query 131: predicted 33649.316934152725, true_card 28260, qerror 1.1907047747400115, latency 0.01130986213684082
estimating query 132: predicted 4612936.330759572, true_card 413771, qerror 11.148524983045142, latency 0.00778961181640625
estimating query 133: predicted 85459.03984021718, true_card 38786, qerror 2.203347595529758, latency 0.008066415786743164
estimating query 134: predicted 73697.91639252544, true_card 67113, qerror 1.0981168535533419, latency 0.0059735774993896484
e

estimating query 201: predicted 1343629.6626671634, true_card 206322, qerror 6.512294678546948, latency 0.008022546768188477
estimating query 202: predicted 1356652.0, true_card 282841, qerror 4.796518185128747, latency 0.004353523254394531
estimating query 203: predicted 7265587.04740976, true_card 3460999, qerror 2.0992745295244983, latency 0.006627798080444336
estimating query 204: predicted 816019.798922775, true_card 371988, qerror 2.1936723736324155, latency 0.006873607635498047
estimating query 205: predicted 14375422.788150525, true_card 1633681, qerror 8.79940624157992, latency 0.00764012336730957
estimating query 206: predicted 4061439.2165994286, true_card 1423047, qerror 2.8540443264343542, latency 0.007791996002197266
estimating query 207: predicted 12068936.036315449, true_card 9976404, qerror 1.2097481253080216, latency 0.0077610015869140625
estimating query 208: predicted 129776.26332387523, true_card 130707, qerror 0.9928792132316956, latency 0.005288362503051758
estim

estimating query 261: predicted 1823754.5470234677, true_card 550289, qerror 3.3141759094284415, latency 0.010401010513305664
estimating query 262: predicted 90551667.86588816, true_card 7661811, qerror 11.8185723800663, latency 0.011861324310302734
estimating query 263: predicted 11504245.110479826, true_card 1236745, qerror 9.302034866104028, latency 0.013058185577392578
estimating query 264: predicted 131186.66249925975, true_card 130349, qerror 1.006426305527927, latency 0.008380413055419922
estimating query 265: predicted 16414512.81217269, true_card 15878293, qerror 1.0337706208200523, latency 0.004538536071777344
estimating query 266: predicted 32211.942873613705, true_card 8538, qerror 3.7727738198188927, latency 0.013519287109375
estimating query 267: predicted 1400201.3243100347, true_card 190399, qerror 7.3540371761933345, latency 0.009804964065551758
estimating query 268: predicted 67130.47200198975, true_card 58524, qerror 1.1470588476862442, latency 0.0068378448486328125


...

estimating query 2312: predicted 1626500.8989155223, true_card 19108639, qerror 0.08511861566464897, latency 0.0284726619720459
estimating query 2313: predicted 1573205.712444947, true_card 168708, qerror 9.325021412410479, latency 0.02603888511657715
estimating query 2314: predicted 2927200.547704915, true_card 44260014, qerror 0.06613645779020574, latency 0.02556467056274414
estimating query 2315: predicted 2738165.1159644364, true_card 377452, qerror 7.254339931870639, latency 0.023833513259887695
estimating query 2316: predicted 19350318.247767933, true_card 2821168, qerror 6.858974101424634, latency 0.02463555335998535
estimating query 2317: predicted 24217073.23003083, true_card 348955301, qerror 0.0693987830551135, latency 0.025860309600830078
estimating query 2318: predicted 5063746.514280159, true_card 3384567, qerror 1.496128312507969, latency 0.02473139762878418
estimating query 2319: predicted 19354936.034237135, true_card 69119371, qerror 0.2800218774305272, latency 0.0257

estimating query 2371: predicted 680482.7341920619, true_card 354404, qerror 1.9200763371521257, latency 0.026567697525024414
estimating query 2372: predicted 2873236.4916523485, true_card 21200677, qerror 0.1355256953187084, latency 0.05172467231750488
estimating query 2373: predicted 513358.49858726375, true_card 246477, qerror 2.0827845948598194, latency 0.04999208450317383
estimating query 2374: predicted 11985267.122781392, true_card 5543342, qerror 2.1621013321533096, latency 0.022640705108642578
estimating query 2375: predicted 59682020.32281356, true_card 4030299, qerror 14.808335640311938, latency 0.039652347564697266
estimating query 2376: predicted 1909930.1946818717, true_card 12304064, qerror 0.15522758941126052, latency 0.025789737701416016
estimating query 2377: predicted 1618801.4442281618, true_card 133378, qerror 12.136944955151238, latency 0.02579522132873535
estimating query 2378: predicted 6137168.495420121, true_card 44869387, qerror 0.13677852330409865, latency 0

estimating query 2429: predicted 172909.0163911762, true_card 28885, qerror 5.986117929415829, latency 0.020388126373291016
estimating query 2430: predicted 6974623.48066115, true_card 3076706, qerror 2.2669125618961155, latency 0.012632608413696289
estimating query 2431: predicted 18385714.871978026, true_card 65723184, qerror 0.27974473774700304, latency 0.023993492126464844
estimating query 2432: predicted 897333.2022508765, true_card 686665, qerror 1.3067990974505421, latency 0.021169662475585938
estimating query 2433: predicted 18384886.648219384, true_card 61160701, qerror 0.30059967180917996, latency 0.028700590133666992
estimating query 2434: predicted 749199.5503475941, true_card 655992, qerror 1.142086413169054, latency 0.024724245071411133
estimating query 2435: predicted 17144089.890318602, true_card 14929017, qerror 1.1483736598544032, latency 0.007072925567626953
estimating query 2436: predicted 98646.45490884902, true_card 38925, qerror 2.534269875628748, latency 0.00794

estimating query 2486: predicted 241248.71379676147, true_card 217851, qerror 1.1074023704126281, latency 0.006299734115600586
estimating query 2487: predicted 166092.2045395408, true_card 154617, qerror 1.0742169654018692, latency 0.00396728515625
estimating query 2488: predicted 1578593.224487227, true_card 451967, qerror 3.492717885348326, latency 0.007774829864501953
estimating query 2489: predicted 36150.81994756329, true_card 37022, qerror 0.9764685848296496, latency 0.01433563232421875
estimating query 2490: predicted 1432288.0, true_card 1331347, qerror 1.0758187009096802, latency 0.004347085952758789
estimating query 2491: predicted 69184.66666666667, true_card 64490, qerror 1.0727968160438313, latency 0.011911869049072266
estimating query 2492: predicted 979842.2275775105, true_card 520351, qerror 1.8830409234872432, latency 0.0067348480224609375
estimating query 2493: predicted 821098.3202898261, true_card 427774, qerror 1.9194675700015105, latency 0.004778385162353516
estim

estimating query 2550: predicted 12005177.262973124, true_card 458451, qerror 26.186391267492326, latency 0.011957883834838867
estimating query 2551: predicted 660164.0314676843, true_card 87842, qerror 7.515357476693203, latency 0.015668630599975586
estimating query 2552: predicted 84291.7430552993, true_card 12124, qerror 6.952469734023367, latency 0.02611541748046875
estimating query 2553: predicted 72704.16635793797, true_card 60963, qerror 1.1925949569072711, latency 0.02983856201171875
estimating query 2554: predicted 331629.5233932236, true_card 172157, qerror 1.926320297131244, latency 0.031106233596801758
estimating query 2555: predicted 12005177.262973124, true_card 438690, qerror 27.365969734831257, latency 0.01587080955505371
estimating query 2556: predicted 1259197.3750851264, true_card 50824, qerror 24.775644874176106, latency 0.026489734649658203
estimating query 2557: predicted 1268340.3823818294, true_card 249157, qerror 5.090526785849201, latency 0.030198335647583008


In [19]:
qerror = np.asarray(qerror)

In [20]:
temp_qerror = copy.deepcopy(qerror)
temp_qerror[temp_qerror < 1] = 1/temp_qerror[temp_qerror < 1]

In [10]:
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(temp_qerror, i)}")
print(f"total inference time: {np.sum(latency)}")

q-error 50% percentile is 3.1804148315503897
q-error 90% percentile is 45.05534055128143
q-error 95% percentile is 134.97948829523907
q-error 99% percentile is 1371.2066796610022
q-error 100% percentile is 438398.8024339036


In [21]:
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(temp_qerror, i)}")
print(f"total inference time: {np.sum(latency)}")

q-error 50% percentile is 2.535956860609279
q-error 90% percentile is 29.43366215743583
q-error 95% percentile is 75.12866071964605
q-error 99% percentile is 1072.511697880098
q-error 100% percentile is 981187.002587944
total inference time: 42.478280782699585


In [22]:
with open("stats_CEB_CE_scheme_200_greedy.txt", "w") as f:
    for p in pred:
        f.write(str(p)+"\n")

In [ ]:
with open("stats_CEB_exec.sql", "r") as f:
    queries = f.readlines()

In [ ]:
with open("stats_CEB_exec.sql", "w") as f:
    for q in queries:
        q = q.split("||")[-1]
        f.write(q)